<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Part-1:-Plotting-points-in-Folium" data-toc-modified-id="Part-1:-Plotting-points-in-Folium-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part 1: Plotting points in Folium</a></span></li><li><span><a href="#Part-2-—-Plotting-polygons-in-Folium-(USGS-dataset)" data-toc-modified-id="Part-2-—-Plotting-polygons-in-Folium-(USGS-dataset)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Part 2 — Plotting polygons in Folium (USGS dataset)</a></span></li></ul></div>

# Visualize Interactive Map

## Introduction

In this note, we want to do a project on wildfires in the US. There are two meain datasets that we use throughout this tutorial: a Kaggle dataset of points stored in a SQLite database containing records for 1.88 million wildfires from 1992 to 2015[1], and a United States Geological Survey (USGS) dataset of polygon data[2] of wildfires from 1878–2019, stored in a shapefile. In this article, we’ll show you how to make an interactive map of this data using open source software (Folium) in Python.

For this project, you’ll need GeoPandas, Folium and Branca (you also need Pandas and Numpy.

## Part 1: Plotting points in Folium

We’ll run through how to plot up points in Folium from the Kaggle dataset. As the data is in a SQLite database, the workflow will be to read the table of interest from the database in to a Pandas dataframe, clean the data up a bit, and then plot the data in Folium for visual analysis.

First, import the necessary libraries, and establish a connection to the sqlite database using sqlite3:

In [ ]:
#import libraries

import sqlite3
import pandas as pd
import geopandas as gpd
import numpy as np
import folium

#connect to sqlite database
conn = sqlite3.connect('data/FPA_FOD_20170508.sqlite')

Then, use the Pandas method ‘read_sql_query’ with a simple SQL statement to select all rows from the ‘Fires’ table. To know which table you are interested in for an analysis, we recommend using [DB Browser for SQLite](https://sqlitebrowser.org/) to briefly analyze the database before making this function call.

In [ ]:
#read the 'wildfires' table in to a pandas dataframe
df = pd.read_sql_query('SELECT * FROM Fires', conn)

pd.set_option('max_columns', None)

df.head()

Next, we create a new dataframe with only the columns we're interested in. Note that alternatively, you could have done this during the last step with a longer SQL statement:

```
df = pd.read_sql_query("""SELECT FIRE_NAME, FIRE_YEAR, 
                       SOURCE_REPORTING_UNIT_NAME,
                       STAT_CAUSE_DESCR, FIRE_SIZE, 
                       LATITUDE, LONGITUDE, STATE
                       FROM Fires""", conn)
```



In [ ]:
#drop most of the columns
df = df[['FIRE_NAME', 'FIRE_YEAR', 
         'SOURCE_REPORTING_UNIT_NAME', 'STAT_CAUSE_DESCR',
         'FIRE_SIZE', 'LATITUDE',
         'LONGITUDE', 'STATE']]

x = len(df)
print(df.shape)
print(df.dtypes)

df.head()

df.shape tells us that this dataset has 1,880,465 rows — displaying this much data on a map will surely be way too much information. So, we decided to create a new dataframe that only contains fires greater than 1000 acres:

In [ ]:
#drop all fires < 1000 acres
df = df.loc[df['FIRE_SIZE'] > 1000]

print(df.shape)

print ('The new dataframe is',
       round(len(df)/x * 100, 3), 
       '% of the original dataset')

This new dataset is only a fraction of the size of the original 1.88 million, containing 11,087 wildfires. Now, we’re ready to make the map! Note that ‘map’ is a Python keyword, so it’s common practice to store the Folium map in a variable ‘m’ , and call ‘m’ to display the map when you’re ready. When dealing with points data, create a variable for each ‘feature group’ that you want to have different characteristics for. You’ll also be able to turn the layer on/off independently from the other layers. We then iterate over each row in the dataframe, and use if statements to populate each of those feature groups.

In [ ]:
m = folium.Map(location = [40.44, -104.81],
               tiles = 'Stamen Terrain',
               zoom_start = 7)

small_wildfires = folium.FeatureGroup(name = '< 10000 Acres')
medium_wildfires = folium.FeatureGroup(name = '10,000 - 50,000 Acres')
large_wildfires = folium.FeatureGroup(name = '50,000 - 100,000 Acres')
xl_wildfires = folium.FeatureGroup(name = '> 100,000 Acres')


for i, v in df.iterrows():
    
    fire_size = float(v['FIRE_SIZE'])
    
    popup = """
    Fire Name : <b>%s</b><br>
    Size (Acres) : <b>%s</b><br>
    State : <b>%s</b><br>
    Cause : <b>%s</b><br>
    Year: <b>%s</b><br>
    """ % (v['FIRE_NAME'], v['FIRE_SIZE'], 
           v['STATE'], v['STAT_CAUSE_DESCR'], 
           v['FIRE_YEAR'])
    
    #small wildfires
    if fire_size < 10000:
        folium.CircleMarker(location = [v['LATITUDE'], 
                                        v['LONGITUDE']],
                           radius = np.log(fire_size) * 0.8,
                           weight = 0,
                           tooltip = popup,
                           color = '#ffeda0',
                           fill_color = '#ffeda0',
                           fill_opacity = 0.7,
                           fill = True).add_to(small_wildfires)
    #medium wildfires    
    if fire_size in range(10000, 50000):
        folium.CircleMarker(location = [v['LATITUDE'], 
                                        v['LONGITUDE']],
                           radius = np.log(fire_size),
                           weight = 0,
                           tooltip = popup,
                           color = '#feb24c',
                           fill_color = '#feb24c',
                           fill_opacity = 0.7,
                           fill = True).add_to(medium_wildfires)
    #large wildfires
    if fire_size in range(50000, 100000):
        folium.CircleMarker(location = [v['LATITUDE'], 
                                        v['LONGITUDE']],
                           radius = np.log(fire_size) * 1.5,
                           weight = 0,
                           tooltip = popup,
                           color = '#fc4e2a',
                           fill_color = '#fc4e2a',
                           fill_opacity = 0.7,
                           fill = True).add_to(large_wildfires)
    #XL wildfires
    if fire_size > 100000:
        folium.CircleMarker(location = [v['LATITUDE'], 
                                        v['LONGITUDE']],
                           radius = np.log(fire_size) * 2,
                           weight = 0,
                           tooltip = popup,
                           color = '##b10026',
                           fill_color = '#b10026',
                           fill_opacity = 0.7,
                           fill = True).add_to(xl_wildfires)

small_wildfires.add_to(m)
medium_wildfires.add_to(m)
large_wildfires.add_to(m)
xl_wildfires.add_to(m)
folium.LayerControl(collapsed = False).add_to(m)

Here’s what the finished map will look like. You can hover and pan around the map to view the wildfires. We used fire acreage as a function of the natural logarithm for the marker size (radius) keyword argument, which makes smaller fires appear as smaller circles and larger fires appear as larger circles. Smaller fires are colored in yellow, while bigger fires are colored in red:

In [ ]:
m

## Part 2 — Plotting polygons in Folium (USGS dataset)

The USGS data comes in a shapefile. We can use GeoPandas to read in the shapefile:

In [ ]:
gdf = gpd.read_file(r'data/Shapefile/US_Wildfires_1878_2019.shp')

print(gdf.shape)
print(gdf.dtypes)

gdf.head()

For this analysis, we're only plotting wildfires greater than 100,000 acres. The reason we do this is because 65,845 polygons generates way too large of a file — even with only wildfires > 100,000 acres (280 polygons), this is still going to be a large html file as our final product. The proper way to deal with this much data would probably be to have the web map pull the data directly from a SQL database. But that would be something to learn later!

In [ ]:
gdf = gdf[['FireName', 'FireYear', 'Acres', 'FireCause', 'Shape_Leng', 'Shape_Area', 'geometry']]
gdf['Acres'] = gdf['Acres'].astype('int')

gdf = gdf.loc[gdf['Acres'] > 100000,]

print(gdf.shape)
gdf.head()

Next, let’s import Branca colormap to generate a linear colormap for our polygons. We scaled the colormap from the size of the smallest fire to the 75th percentile of fire size (‘Acres’). This way, the largest 25% of wildfires will all be colored red, while most of the fires (smallest 75%) will exhibit a wide range of colors. We also defined a quick function to reverse the colormap. To view all available colormap options, call ‘cm.linear’.

In [ ]:
import branca.colormap as cm

fire_min = gdf['Acres'].min()
fire_max = gdf['Acres'].max()
fire_75 = gdf['Acres'].quantile(q = 0.75)

linear = cm.linear.RdYlBu_08.scale(fire_min, fire_75)

def reversed_colormap(existing):
    return cm.LinearColormap(
        colors=list(reversed(existing.colors)),
        vmin=existing.vmin, vmax=existing.vmax)

linear = reversed_colormap(linear)

Now we’re ready to plot up the polygons. For the colormap, you’ll need to make a dictionary with ‘FireName’ as the keys and ‘Acres’ as the values. We use the dictionary values as an argument in the lambda function for the variable ‘map colors’, and then pass ‘map_colors’ as a keyword argument for the style function in the GeoJson folium method.


In [ ]:
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

m = folium.Map(
    location = [39, -105],
    tiles = 'Stamen Terrain',
    zoom_start = 6)

fire_dict = dict(zip(gdf['FireName'], gdf['Acres']))

tooltip = GeoJsonTooltip(
    fields = ['FireName', 'FireYear', 
              'Acres', 'FireCause'],
    aliases = ['Name', 'Year', 'Acres', 'Cause'],
    #localize = True,
    sticky = True,
    labels = True,
    style = """
    background-color: #F0EFEF;
    border: 2px solid black;
    border-radius: 3px,
    box-shadow: 3px;
    """,
    max_width = 800
)

    
map_colors = lambda x: {
    'fillColor': linear(fire_dict[x['properties']['FireName']]),
    'color': 'black',
    'weight': 0.25,
    'fillOpacity': 0.5
}

#define a function for map_colors, if you prefer
# def map_colors(x):
#     return {'opacity': 1,
#             'weight': 0.5,
#             'fillColor': linear(fire_dict[x['properties']['FireName']]),
#             'color':'black'}


folium.GeoJson(
    gdf,
    name = 'Wildfires',
    style_function = map_colors,
    tooltip = tooltip
).add_to(m)

folium.LayerControl(collapsed = False).add_to(m)

And, we now have polygons!

In [ ]:
m

IF this is too big to be displayed in your machine, try saving the map as html and then embed it in your notebook or open the html file in a browser

In [ ]:
m.save(r'data/USGS_wildfires.html')

In [ ]:
from IPython.core.display import display, HTML

display(HTML('data/USGS_wildfires.html'))